In [1]:
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark import SparkContext, SparkConf,StorageLevel
# create sparksession
conf = SparkConf().setAppName("MLLab").setMaster("local[2]")
sc = SparkContext(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [37]:
features_path = "../data/features.csv"
labels_path = "../data/labels.csv"

In [38]:
features = spark.read.format("csv").option("inferSchema", 'true').option("header", "true").load(features_path)
labels = spark.read.format("csv").option("inferSchema", 'true').option("header", "true").load(labels_path)

In [78]:
features.columns

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [79]:
from pyspark.sql.functions import col
full_train = features.alias('a').join(labels.alias('b'),col('b.id') == col('a.id')).drop('id')

In [80]:
full_train.take(1)

[Row(amount_tsh=6000.0, date_recorded=datetime.datetime(2011, 3, 14, 0, 0), funder='Roman', gps_height=1390, installer='Roman', longitude=34.93809275, latitude=-9.85632177, wpt_name='none', num_private=0, basin='Lake Nyasa', subvillage='Mnyusi B', region='Iringa', region_code=11, district_code=5, lga='Ludewa', ward='Mundindi', population=109, public_meeting=True, recorded_by='GeoData Consultants Ltd', scheme_management='VWC', scheme_name='Roman', permit=False, construction_year=1999, extraction_type='gravity', extraction_type_group='gravity', extraction_type_class='gravity', management='vwc', management_group='user-group', payment='pay annually', payment_type='annually', water_quality='soft', quality_group='good', quantity='enough', quantity_group='enough', source='spring', source_type='spring', source_class='groundwater', waterpoint_type='communal standpipe', waterpoint_type_group='communal standpipe', status_group='functional')]

In [81]:
full_train.schema

StructType(List(StructField(amount_tsh,DoubleType,true),StructField(date_recorded,TimestampType,true),StructField(funder,StringType,true),StructField(gps_height,IntegerType,true),StructField(installer,StringType,true),StructField(longitude,DoubleType,true),StructField(latitude,DoubleType,true),StructField(wpt_name,StringType,true),StructField(num_private,IntegerType,true),StructField(basin,StringType,true),StructField(subvillage,StringType,true),StructField(region,StringType,true),StructField(region_code,IntegerType,true),StructField(district_code,IntegerType,true),StructField(lga,StringType,true),StructField(ward,StringType,true),StructField(population,IntegerType,true),StructField(public_meeting,BooleanType,true),StructField(recorded_by,StringType,true),StructField(scheme_management,StringType,true),StructField(scheme_name,StringType,true),StructField(permit,BooleanType,true),StructField(construction_year,IntegerType,true),StructField(extraction_type,StringType,true),StructField(extr

In [119]:
from pyspark.sql.types import DoubleType, IntegerType, DateType, BooleanType

double_columns = ['amount_tsh', 'longitude', 'latitude', ]
int_columns = ['gps_height', 'num_private', 'region_code', 'district_code', 'population','construction_year']
bool_columns = ['public_meeting','permit']
numeric_columns = double_columns + int_columns
date_columns = ['date_recorded']

indexed_columns = date_columns + ['lga']
drop_colums =  ['ward', 'funder', 'installer', 'wpt_name', 'subvillage', 'scheme_name', 'recorded_by']

target_column = 'status_group'

not_unique_columns = double_columns + int_columns + bool_columns + drop_colums + [target_column] + indexed_columns
unique_columns = [elem for elem in full_train.columns if elem not in not_unique_columns] + bool_columns

In [120]:
dropped = full_train#.drop('date_recorded')

In [121]:
for column in numeric_columns:
    dropped =  dropped.withColumn(column, dropped[column].cast("double"))

In [122]:
for column in bool_columns:
    dropped =  dropped.withColumn(column, dropped[column].cast("string"))

In [123]:
for column in date_columns:
    dropped =  dropped.withColumn(column, dropped[column].cast("string"))


In [124]:
unique_values = {}
print("unique_columns:")
for column in unique_columns:
    unique_values[column] = dropped.select(column).distinct().count()
    print(column, unique_values[column])
print('\ndropped columns:')
for column in drop_colums:
    unique_values[column] = full_train.select(column).distinct().count()
    print(column, unique_values[column])
print('\nindexed columns:')
for column in indexed_columns:
    unique_values[column] = full_train.select(column).distinct().count()
    print(column, unique_values[column])

unique_columns:
basin 9
region 21
scheme_management 13
extraction_type 18
extraction_type_group 13
extraction_type_class 7
management 12
management_group 5
payment 7
payment_type 7
water_quality 8
quality_group 6
quantity 5
quantity_group 5
source 10
source_type 7
source_class 3
waterpoint_type 7
waterpoint_type_group 6
public_meeting 3
permit 3

dropped columns:
ward 2092
funder 1898
installer 2146
wpt_name 37400
subvillage 19288
scheme_name 2697
recorded_by 1

indexed columns:
date_recorded 356
lga 125


In [125]:
dropped.schema

StructType(List(StructField(amount_tsh,DoubleType,true),StructField(date_recorded,StringType,true),StructField(funder,StringType,true),StructField(gps_height,DoubleType,true),StructField(installer,StringType,true),StructField(longitude,DoubleType,true),StructField(latitude,DoubleType,true),StructField(wpt_name,StringType,true),StructField(num_private,DoubleType,true),StructField(basin,StringType,true),StructField(subvillage,StringType,true),StructField(region,StringType,true),StructField(region_code,DoubleType,true),StructField(district_code,DoubleType,true),StructField(lga,StringType,true),StructField(ward,StringType,true),StructField(population,DoubleType,true),StructField(public_meeting,StringType,true),StructField(recorded_by,StringType,true),StructField(scheme_management,StringType,true),StructField(scheme_name,StringType,true),StructField(permit,StringType,true),StructField(construction_year,DoubleType,true),StructField(extraction_type,StringType,true),StructField(extraction_type

In [126]:
from pyspark.sql.functions import avg
def fill_with_mean(df, include=set()): 
    stats = df.agg(*(
        avg(c).alias(c) for c in include
    ))
    return df.fillna(stats.first().asDict())

def fill_with_median(df, include=list()):
    medians = df.approxQuantile(include, [0.5], 0.01)
    assert isinstance(include, (tuple,list))
    medians = list(map(lambda x: x[0],medians))
    medians=dict(zip(include,medians))
    print(medians)
    return df.fillna(medians)


dropped = fill_with_median(dropped, numeric_columns)

{'amount_tsh': 0.0, 'longitude': 34.87524907, 'latitude': -5.06903933, 'gps_height': 353.0, 'num_private': 0.0, 'region_code': 12.0, 'district_code': 3.0, 'population': 20.0, 'construction_year': 1985.0}


In [127]:
features_column = "features"
label_column = "label"

In [128]:
dropped.select(numeric_columns).take(5)

[Row(amount_tsh=6000.0, longitude=34.93809275, latitude=-9.85632177, gps_height=1390.0, num_private=0.0, region_code=11.0, district_code=5.0, population=109.0, construction_year=1999.0),
 Row(amount_tsh=0.0, longitude=34.6987661, latitude=-2.14746569, gps_height=1399.0, num_private=0.0, region_code=20.0, district_code=2.0, population=280.0, construction_year=2010.0),
 Row(amount_tsh=25.0, longitude=37.46066446, latitude=-3.82132853, gps_height=686.0, num_private=0.0, region_code=21.0, district_code=4.0, population=250.0, construction_year=2009.0),
 Row(amount_tsh=0.0, longitude=38.48616088, latitude=-11.15529772, gps_height=263.0, num_private=0.0, region_code=90.0, district_code=63.0, population=58.0, construction_year=1986.0),
 Row(amount_tsh=0.0, longitude=31.13084671, latitude=-1.82535885, gps_height=0.0, num_private=0.0, region_code=18.0, district_code=1.0, population=0.0, construction_year=0.0)]

In [129]:
dropped.schema
from pyspark.sql.functions import isnan, when, count, col
temp_df = dropped.select([target_column])
temp_df.select([count(when(isnan(c), c)).alias(c) for c in temp_df.columns]).show()

+------------+
|status_group|
+------------+
|           0|
+------------+



In [130]:
dropped.columns

['amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group',
 'status_group']

In [131]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

encoders = []
for column in unique_columns + indexed_columns:
    enc = StringIndexer(inputCol=column,
                                 outputCol="_"+column)
    enc.setHandleInvalid('keep')
    encoders.append(enc)

new_indexed_columns = list(map(lambda x: "_"+x, indexed_columns))
new_unique_columns = list(map(lambda x: "_"+x, unique_columns))
new_new = list(map(lambda x: "_"+x, new_unique_columns))
encoder2 = OneHotEncoderEstimator(inputCols=new_unique_columns,
                                 outputCols=new_new)


#encoder3 = OneHotEncoderEstimator(inputCols=bool_columns,
#                                 outputCols=list(map(lambda x: "_"+x, bool_columns)))

print(new_indexed_columns, unique_columns)

label_encoder = StringIndexer(inputCol=target_column,
                                 outputCol=label_column)

assembler = VectorAssembler(
    inputCols=(numeric_columns + new_new + new_indexed_columns),
    outputCol=features_column)



pipeline = Pipeline(stages=[*encoders, encoder2, label_encoder, assembler])


model = pipeline.fit(dropped)
encoded = model.transform(dropped)

preprocessed_data = encoded.drop(*(new_unique_columns+unique_columns + bool_columns))


['_date_recorded', '_lga'] ['basin', 'region', 'scheme_management', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'public_meeting', 'permit']


In [133]:
preprocessed_data.show(1)

+----------+-------------------+------+----------+---------+-----------+-----------+--------+-----------+----------+-----------+-------------+------+--------+----------+--------------------+-----------+-----------------+------------+--------------+----+--------------+-------------+-----------------------+--------------+--------------+-----------------+----------------+-------------+-------------+-------------+-----------------------+---------------+---------------+--------------+-----------------+-------------+----------------+------------------+--------------+-----------------------+-------------------+-----+--------------------+
|amount_tsh|      date_recorded|funder|gps_height|installer|  longitude|   latitude|wpt_name|num_private|subvillage|region_code|district_code|   lga|    ward|population|         recorded_by|scheme_name|construction_year|status_group|_date_recorded|_lga|  __management|    __payment|__extraction_type_class|      __region|__source_class|__extraction_type|__publi

In [134]:
preprocessed_data.select(features_column).take(1)

[Row(features=SparseVector(183, {0: 6000.0, 1: 34.9381, 2: -9.8563, 3: 1390.0, 5: 11.0, 6: 5.0, 7: 109.0, 8: 1999.0, 15: 1.0, 18: 1.0, 39: 1.0, 51: 1.0, 69: 1.0, 82: 1.0, 89: 1.0, 101: 1.0, 111: 1.0, 118: 1.0, 120: 1.0, 128: 1.0, 134: 1.0, 139: 1.0, 144: 1.0, 154: 1.0, 161: 1.0, 164: 1.0, 171: 1.0, 177: 1.0, 180: 1.0, 181: 3.0, 182: 37.0}))]

In [135]:
preprocessed_data.select([features_column, label_column]).count()

59400

In [136]:
from pyspark.sql.functions import isnan, when, count, col
temp_df = preprocessed_data.select([label_column])
temp_df.select([count(when(isnan(c), c)).alias(c) for c in temp_df.columns]).show()

+-----+
|label|
+-----+
|    0|
+-----+



In [138]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = preprocessed_data.select([features_column, label_column]).randomSplit([0.8, 0.2])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol=label_column, featuresCol=features_column)
dt.setMaxBins(400)
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", label_column, features_column).show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol=label_column, predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

print("Accuracy = %g " % (accuracy))

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(183,[0,1,2,3,4,5...|
|       0.0|  2.0|(183,[0,1,2,3,4,5...|
|       0.0|  0.0|(183,[0,1,2,3,4,5...|
|       0.0|  0.0|(183,[0,1,2,3,4,5...|
|       0.0|  0.0|(183,[0,1,2,3,4,5...|
+----------+-----+--------------------+
only showing top 5 rows

Test Error = 0.285245 
Accuracy = 0.714755 
